In [1]:
# for loading/processing the images
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils  import load_img
from tensorflow.keras.utils import img_to_array

# models
from keras.applications.vgg16 import VGG16
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

2023-09-23 10:11:02.637024: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-23 10:11:02.638230: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 10:11:02.664550: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 10:11:02.665133: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 10:11:03.201001: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

2023-09-23 10:12:02.995126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-23 10:12:02.995435: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


553467096/553467096 [==============================] - 170s 0us/step


In [23]:



def get_labels_and_paths(input_folder):
   return [(dp.split("/")[1],os.path.join(dp, f)) for dp, dn, filenames in os.walk('input') for f in filenames if os.path.splitext(f)[1] == '.jpg']  


def extract_features(file, model):
    # levanta imagen como array 224x224
    img = load_img(file, target_size=(224,224))
    # convierte img a numpy array (originalmente es 'PIL.Image.Image')
    img = np.array(img)
    # reshape para tener formato necesario para el modelo (num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3)
    # prepara imagen para modelo (función de keras)
    imgx = preprocess_input(reshaped_img)
    # extrae features
    if isinstance(imgx, (np.ndarray, np.generic) ):
        features = model.predict(imgx, use_multiprocessing=True)
        return features
    else:
        raise ValueError(f"No se ha logrado procesar input {file}")

def preprocess(names_and_paths:list, model:Model)->np.array:
    preprocessed_data = {}
    for name, path in names_and_paths:
      featuress = extract_features(path, model)
      preprocessed_data[path] = {'label':name,
                    'features':featuress}
      print(f"Extracción features de {path}")
    return preprocessed_data


# function that lets you view a cluster (based on identifier)
def view_cluster(gps,cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = gps
    # only allow up to 30 images to be shown at a time
    if len(files) > 10:
        print(f"Clipping cluster size from {len(files)} to 10")
        files = files[:9]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(1,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.title('Cluster n:' + str(cluster))
        plt.axis('off')




In [15]:
names_and_paths = get_labels_and_paths(input_folder='input')

('Jasmine', 'input/Jasmine/Jasmine (7576).jpg')

In [36]:
ejemplo = 'input/Jasmine/Jasmine (7576).jpg'

# levanta imagen como array 224x224
img = load_img(ejemplo, target_size=(224,224)) # Te la lleva al formato que necesitás. 
# convierte img a numpy array (originalmente es 'PIL.Image.Image')
img = np.array(img)
# reshape para tener formato necesario para el modelo (num_of_samples, dim 1, dim 2, channels)
reshaped_img = img.reshape(1,224,224,3)
# prepara imagen para modelo (función de keras)
imgx = preprocess_input(reshaped_img)

import cv2

def img_to_numpy(path): 
    img = cv2.imread(filename=path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


import matplotlib.pyplot as plt

# img_to_numpy(ejemplo).shape
img.shape

# # extrae features
# if isinstance(imgx, (np.ndarray, np.generic) ):
#     features = model.predict(imgx, use_multiprocessing=True)
#     return features

(224, 224, 3)

In [24]:
data_dict = preprocess(names_and_paths=names_and_paths, model=Model)

TypeError: Model.predict() missing 1 required positional argument: 'x'